<a href="https://colab.research.google.com/github/ev1025/1day1commit/blob/main/%EC%BD%94%EB%93%9C%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%B8%A0/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%B4%9D%EC%A0%95%EB%A6%AC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 하이퍼파라미터   
!pip3 install hyperopt



### Exhaustive Gride Search
- 범위로 설정한 모든 값으로 탐색진행
- 하이퍼파라미터 값 3 x 3 x n(cv값) 회 진행
- 튜닝할 하이퍼 파라미터의 수가 많을때는 사용하기 효율적이지 않다.


In [ ]:
params = {
    "xgbclassifier__max_depth" : [2,4,6]                 # "모델__하이퍼파라미터" : 탐색할 값 목록
    "xgblcassifier__colsample_bytree" : [0.2, 0.4, 0.8],
}

In [ ]:
from sklearn.mdoel_selection import GriedSearchCV

gird = GridSearchCV(모델,
                    param_grid = params,  # 미리 지정해둔 파라미터 범위리스트
                    scoring = 'roc_auc',  # 사용할 점수
                    cv = n,               # 검증할 횟수
                    verbose = n           # 검증결과 얼마나 상세하게 볼지(0~3, 3이 제일 상세하게)
)
grid.fit(X_train, y_train)

grid.best_params_    # 최적의 하이퍼파라미터
grid.best_score_     # 최적의 검증 점수
gird.cv_results_      # 검증 결과 목록

### Randomized Search
- 범위로 설정한 값에서 무작위로 추출(횟수는  n_iter x cv의 수만큼만 진행)
- 범위가 넓고 빠르지만 모든 조합에 대해 검증하지 않아 가장 이상적인 조합을 발견하지 못할 수 있다.
- scipy.stats로 범위를 분포로 지정도 가능


In [ ]:
from scipy.stats.distributions import uniform
from sklearn.impute import SimpleImputer

params = {
    "simpleimpouter__strategy" : ['median', 'mean']                 # 인코딩, 결측치처리도 가능
    "xgbclassifier__colsample_bytree" : quniform(0,1, 1, 0.1)       # 범위내에서 step으로 선택 
    "xgbclassifier__colsample_bytree" : uniform(loc=0.5, scale=0.5) # 0.5 ~ 1 사이에서 랜덤추출
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

randomized = RandomizedSearchCV(
    모델,
    param_distributioon=params,  # 조정할 하이퍼파라미터
    scoring='roc_auc',         
    n_iter = n,                  # 돌릴 횟수
    cv = n,                      # 검증할 횟수
    verbose=n,
    random_state=n,
)
randomized.fit(X_train, y_train)

randomized.best_params_   # 최적의 하이퍼파라미터(찾은 값)
randomized.best_score_    # 최적의 검증점수

randomized.cv_results_    # 하이퍼파라미터 탐색 리스트

### Bayesian Search
- 검증할 하이퍼파라미터 범위내에서, 이전에 탐색한 조합중 성능이 좋은 조합을 중심으로 확률적으로 탐색
- hyperopt 라이브러리로 베지안 서치를 할 수 있다.(단, hyperopt에서 제공하는 범위를 사용해야함)
- hyperopt의 fmin는 최종적으로 loss를 가장 작게 만드는 파라미터 조합을 선택(함수를 만들어서 loss와 STATUS값을 제공해야함)
- loss이므로 클수록 좋은 함수는 - 부호를 붙여주어야함
- fmin의 max_evals 수 만큼 하이퍼파라미터 탐색

In [ ]:
# hyperopt 범위 옵션
hp.choice('하이퍼파라미터', list or tuple) # 리스트나 튜플의 값을 선택하여 탐색
hp.randint('하이퍼파라미터', upper)        #  0 ~ upper 사이의 정수를 랜덤으로 선택
hp.uniform('하이퍼파라미터',low, high )    # low ~ high 범위 내의 실수를 랜덤으로 선택
hp.quniform('하이퍼파라미터', low, high, q)# low ~ high 사이 균등분포에서 q간격의 지점에서 랜덤수 선택
hp.normal('하이퍼파라미터', mu, sigma)     # mu(평균), sigma(표준편차)를 갖는 정규분포 모델에서 실수를 랜덤선택

In [ ]:
# Bayesian Search순서 : 1. 튜닝할 params작성 / 2.파이프라인구축 함수 / 3. 학습 및 검증 함수 / 4. fmin으로 loss값 받아서 베이지안서치 
from hyperopt import hp

params= {
    "simpleimputer__strategy" : hp.choice('strategy', ['mean', 'median']), # 목록에서 랜덤선택
    "xgbclassifier__max_depth" : hp.quniform("max_depth", 2, 10, 2)        # 2~10 사이에서 2 간격으로 랜덤선택
    "xgblcassifier__colsample_bytree" : hp.uniform("colsample_bytree", 0.5, 1.0) # 0.5~1.0 사이에서 실수 랜덤선택
}

In [ ]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.model.selection import corss_val_score
from sklearn.pipeline import make_pipeline
import numpy as np

def get_pipe(params): # 파이프라인 구축 함수
    params['xgbclassifier__max_depth'] = int(params['xgbclassifier_max_depth']) # 정수형 파라미터로 변경

    pipe = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(),
        XGBClassifier(
            random_state = 2,
            objective = 'binary:logistic',
            eval_metric = 'error',
            n_estimators = 20,
            learning_rate = 0.2,

        ),
    )
    pipe = pipe.set_params(**params) # parmas의 값을 넣어줌
    return pipe


def fit_and_eval(params):      # 학습 및 검증 함수
    pipe = get_pipe(params)    # 모델 생성
    score = cross_val_score(   # 점수 검증
        pipe,
        X_train,
        y_train,
        cv = n,
        scoring='roc_auc',
    )
    avg_cv_score = -np.mean(score) # loss값을 구해야 하므로 높을수록 좋은 값은 -를 붙여줌
    return {"loss": avg_cv_score, "status : STATUS_OK"}  # fmin에 loss와 STATUS 제공

trials = (Trials()) # fmin의 trials에 넣어 실행하면 fit의 학습결과를 모두 저장(grid, randomize의 .cv_results_)

best_params = fmin(    # hyperopt의 fmin은 loss정보를 얻어서 최적의 하이퍼파라미터 탐색
    fn = fit_and_eval, # 학습, 검증 함수
    trials = trials,   # 결과를 저장할 객체
    space = params,    # 하이퍼파라미터 범위
    algo = tpe.suggest,# suggest 알고리즘 사용
    max_evals = n      # 탐색 횟수    
)

In [ ]:
trials.trials # trials에 하이퍼파라미터 탐색 결과 

In [ ]:
trials.best_trial['misc']['vals']    # 최적의 하이퍼파라미터 구성(tiral 목록의 misc의 vals에 저장되어 있음)
-trials.best_trial['result']['loss'] # 최적의 하이퍼파라미터의 검증점수(앞에서 - 붙였으니 다시 -붙여줌)

## 특성선택


### 1. 직관기반 선택
- 데이터가 충분할 때는 예측에 중요하지 않은 특성이 학습데이터에 포함되어 있어도 모델성능에 크게 관여하지 않음(데이터가 적으면 노이즈 과적합이 발생)
- 개발자가 주요하지 않은 특성을 선제적으로 배제(추천 알고리즘에서 사용자ID, 이름 등 제거)

### 2. 특성중요도기반 선택
- 각 특성의 중요도에 따라 특성을 선택
- 회귀모델 : model.coef_
- 분류모델 : model.feature_importances_

### 3. 통계량 기반 특성 선택
- 피어슨 상관계수 : 선형성 타겟의 상관관계를 추측(np.corrcoef)
- 스피어만 상관계수 : 단조성 타겟의 상관계수를 추측(spearmanr)
- SeletKBest : 회귀 or 분류 통계량에 따른 분류
   - 분류타겟 옵션 : f_classif / mutual_info_classif / chi2
   - 회귀타겟 옵션 : f_regression / mutual_info_regression


In [ ]:
from scipy.stats import spearmanr

np.corrcoef(x,y)[0, 1] # 특성 x,y의 선형성 점수
spearmanr(x,y)         # 특성 x,y의 단조성 점수

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif 

selector = SelectKBest(score_func = f_classif, k=n)          # 분류문제, 상위 n개의 특성만 사용
X_train_selected = selector.fit_transform(X_train, y_train)  # 선택된 n개의 특성으로 훈련데이터 생성
X_val_selected = seleector.transform(X_val)                  # 사용할 모델에 model.fit(X_val_selected, y_val)

In [ ]:
X_train.columns[selector.get_support()].tolist()             # SelectKBest에서 선택된 특성 출력